# 1. Business Task
Cyclistic is a company looking for how to keep growing in the future. Their business is based on different bike rental plans:

-Single-ride pass.

-Full-day pass.

-Annual membership.

We are going to analyse how annual members (Annual membership) differ from casual riders (Single pass and Full-day pass) using Cyclistic, in order to help the organization design marketing strategies aimed at converting casual riders into annual members. 

Annual members are more profitable than casual riders, so stakeholders believe that increasing the amount of annual riders is key for Cyclistic's growth in the future.

# 2. Data Preparation

The data used is internally collected monthly data from 2013 to 2022 by Cyclistic from their users. The data is stored by the company on AWS' S3 cloud object storaged solution in dot zip files.

https://divvy-tripdata.s3.amazonaws.com/index.html

In general the data contains anonymous user information regarding:

-ride ID

-starting station

-ending station

-user type (casual rider or annual member)

It's stored in the structured data format. However, data was not saved the same throught the years.










#### Storing the data
1. The zip files were uncompressed, in an individual carpet for each time period, in the 'csv files' folder.

2. The csv files containg information about the Byke Trips were copied to the 'raw data' folder.



### Cleaning the data
There's some cleaning and formatting that needs to be done to the data because of the following issues:

-There's columns missing from one time period to the other, such as: 

###### user gender, birthyear, start_lat, start_lng, end_lat, end_lng, rideable_type, bikeid, tripduration.

-The following columns are the same although they have different names:

###### trip_id == 01 - Rental Details Rental ID == ride_id

###### starttime == 01 - Rental Details Local Start Time == started_at

###### stoptime == 01 - Rental Details Local End Time  == ended_at

###### from_station_name == 03 - Rental Start Station Name == start_station_name

###### to_station_name == 02 - Rental End Station Name == end_station_name

###### from_station_id == start_station_id == 03 - Rental Start Station ID

###### to_station_id == 02 - Rental End Station ID == end_station_id

###### usertype == User Type == member_casual

###### tripduration == 01 - Rental Details Duration In Seconds Uncapped

###### bikeid = 01 - Rental Details Bike ID

###### gender	== Member Gender

###### birthyear == 05 - Member Details Member Birthday Year



#### Understanding the data
- From 2013 until April 2020 the "Trips" table only contained the following columns:

###### trip_id	starttime,	stoptime,	bikeid,	tripduration	from_station_id	from_station_name,	to_station_id,	to_station_name	usertype,	gender,	birthyear

Information regarding geographical points (start_lat, start_lng, end_lat, end_lng) present in the Trips table from April 2020 till now, were from 2013 to 2013 until 2018 Q1 previously stored in separate tables  whose title contained the word 'Stations'.

- Data collected from 2018 Q1 until 2020 Q1 doesn't contain information regarding geographical points (columns: start_lat, start_lng, end_lat, end_lng) about the stations. There isn't a "Stations" table, nor does the "Trips" table contain such columns.

- 2018 Q1 has the trip_id, start_time, and end_time columns missing.

- User Type categories 'Customer' and 'casual' are the same, as well as 'Subscriber' and 'member'.

- In some tables the datetime structure follows a 12 hour AM/PM period, and in others a 24 hour period is used.

#### Data cleaning and formatting with Python and Pandas
The following changes were performed to the data